In [54]:
import numpy as np
import pandas as pd
import seaborn as sns
import wrangle
from talos.model import lr_normalizer
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.models import Sequential
import talos

In [55]:
multiplier = 1000
df = pd.read_csv('data/cclebeta.csv', index_col=0)
display(df.head())

columns = df.columns.tolist()
columns.insert(len(columns), columns.pop(0))
data = df[columns]
display(data.head())

data['Beta'] = data['Beta'].apply(lambda x: x * multiplier)
data.head()

,Beta,1,2,9,10,12,13,14,15,16,...,100628307,100630918,100631378,100642175,100652770,100652791,100653022,100689073,100689074,100820829
22rv1,5.625000e-08,4.423446,4.134148,10.505010,4.180319,3.828985,3.416169,8.924422,3.547737,10.489760,...,3.641889,5.662483,3.535540,5.441015,4.484159,3.272955,3.932815,3.518021,4.202408,4.675101
42mgba,2.409000e-02,6.856747,3.792826,8.117941,4.300842,3.534361,3.465385,8.689804,3.730495,10.063663,...,3.527367,4.894637,3.717672,4.600157,3.986362,3.102421,4.577944,3.650426,3.155024,4.213095
5637,9.217000e-03,3.817624,3.769035,7.944338,4.000219,4.793569,4.246643,9.218668,3.775753,8.843498,...,3.502305,5.375774,3.438822,5.457984,5.054120,3.542372,4.057933,3.664296,3.268805,7.120733
647v,6.135000e-02,4.322558,3.730195,8.515086,4.421911,3.894124,3.649778,8.962272,3.469970,10.341420,...,3.402495,5.286006,3.415911,4.769920,4.744037,3.426386,4.216155,3.417696,3.374419,6.135024
769p,1.736000e-02,4.020103,3.846175,8.278318,4.400144,4.855821,3.731236,9.109365,3.576632,10.172053,...,3.720749,5.318118,3.583233,4.643916,4.882646,3.490162,4.041154,3.375546,3.291270,4.529068


,1,2,9,10,12,13,14,15,16,18,...,100630918,100631378,100642175,100652770,100652791,100653022,100689073,100689074,100820829,Beta
22rv1,4.423446,4.134148,10.505010,4.180319,3.828985,3.416169,8.924422,3.547737,10.489760,5.448634,...,5.662483,3.535540,5.441015,4.484159,3.272955,3.932815,3.518021,4.202408,4.675101,5.625000e-08
42mgba,6.856747,3.792826,8.117941,4.300842,3.534361,3.465385,8.689804,3.730495,10.063663,7.104628,...,4.894637,3.717672,4.600157,3.986362,3.102421,4.577944,3.650426,3.155024,4.213095,2.409000e-02
5637,3.817624,3.769035,7.944338,4.000219,4.793569,4.246643,9.218668,3.775753,8.843498,3.573155,...,5.375774,3.438822,5.457984,5.054120,3.542372,4.057933,3.664296,3.268805,7.120733,9.217000e-03
647v,4.322558,3.730195,8.515086,4.421911,3.894124,3.649778,8.962272,3.469970,10.341420,6.962116,...,5.286006,3.415911,4.769920,4.744037,3.426386,4.216155,3.417696,3.374419,6.135024,6.135000e-02
769p,4.020103,3.846175,8.278318,4.400144,4.855821,3.731236,9.109365,3.576632,10.172053,4.266400,...,5.318118,3.583233,4.643916,4.882646,3.490162,4.041154,3.375546,3.291270,4.529068,1.736000e-02


,1,2,9,10,12,13,14,15,16,18,...,100630918,100631378,100642175,100652770,100652791,100653022,100689073,100689074,100820829,Beta
22rv1,4.423446,4.134148,10.505010,4.180319,3.828985,3.416169,8.924422,3.547737,10.489760,5.448634,...,5.662483,3.535540,5.441015,4.484159,3.272955,3.932815,3.518021,4.202408,4.675101,0.000056
42mgba,6.856747,3.792826,8.117941,4.300842,3.534361,3.465385,8.689804,3.730495,10.063663,7.104628,...,4.894637,3.717672,4.600157,3.986362,3.102421,4.577944,3.650426,3.155024,4.213095,24.090000
5637,3.817624,3.769035,7.944338,4.000219,4.793569,4.246643,9.218668,3.775753,8.843498,3.573155,...,5.375774,3.438822,5.457984,5.054120,3.542372,4.057933,3.664296,3.268805,7.120733,9.217000
647v,4.322558,3.730195,8.515086,4.421911,3.894124,3.649778,8.962272,3.469970,10.341420,6.962116,...,5.286006,3.415911,4.769920,4.744037,3.426386,4.216155,3.417696,3.374419,6.135024,61.350000
769p,4.020103,3.846175,8.278318,4.400144,4.855821,3.731236,9.109365,3.576632,10.172053,4.266400,...,5.318118,3.583233,4.643916,4.882646,3.490162,4.041154,3.375546,3.291270,4.529068,17.360000


In [94]:
X = data.drop(['Beta'], axis=1)
y = data['Beta']

In [96]:
# Train - Test
X_trainval, X_test, y_trainval, y_test = train_test_split(X, y, test_size=0.05, random_state=69)
# Split train into train-val
X_train, X_val, y_train, y_val = train_test_split(X_trainval, y_trainval, test_size=0.05, random_state=21)

In [97]:
autom8 = talos.autom8.AutoScan('binary', 5)

In [99]:
autom8.start(x=X_train,
             y=y_train,
             x_val=X_val,
             y_val=y_val,
             fraction_limit=0.000001)

TypeError: can only concatenate str (not "int") to str